In [3]:
import uuid
import os
INPUT_DIR = './data/'
OUTPUT_DIR = './roberta/'+str(uuid.uuid1())+'/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

from shutil import copyfile
import gc
import re
import ast
import sys
import copy

import json
import time
import math
import shutil
import string
import pickle
import random
import itertools
from pathlib import Path
import warnings
from FGM import FGM
os.environ["TOKENIZERS_PARALLELISM"] = "true"
warnings.filterwarnings("ignore")
import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from model import CustomModel
from CFG_roberta import CFG
from utils import get_cpc_texts, get_logger, get_score, seed_everything, AverageMeter, asMinutes, timeSince
from data_utils import prepare_input, TrainDataset

device = torch.device('cuda:0')


In [2]:
train = pd.read_csv(INPUT_DIR+'train.csv')
cpc_texts = get_cpc_texts()
torch.save(cpc_texts, OUTPUT_DIR+"cpc_texts.pth")
train['context_text'] = train['context'].map(cpc_texts)


In [3]:
context_text = list(train['context_text'])

In [4]:
context_text_list = []

for context_text_item in context_text:
    tmp_phrases = context_text_item.split('.')
    phrases = []
    for tmp_phrase in tmp_phrases:
        tmp_p = tmp_phrase.split(';')
        for itm in tmp_p:
            phrases.append(itm.strip(' '))
    context_text_list.append(phrases)

In [5]:
train = train.drop(['context'],axis=1).drop(['id'],axis=1)

In [6]:
raw_cnt = train.shape[0]

In [7]:
train

,anchor,target,score,context_text
0,abatement,abatement of pollution,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
1,abatement,act of abating,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
2,abatement,active catalyst,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
3,abatement,eliminating process,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
4,abatement,forest region,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
...,...,...,...,...
36468,wood article,wooden article,1.00,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...
36469,wood article,wooden box,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...
36470,wood article,wooden handle,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...
36471,wood article,wooden material,0.75,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...


In [8]:
failure = 0
for i in range(raw_cnt):
    if i%5000 == 0:
        print(i)
    len_words = len(context_text_list[i])
    if len_words < 2:
        failure += 1
    random.shuffle(context_text_list[i])
    context_generated = ''
    for j in range(len_words):
        context_generated += context_text_list[i][j]
        context_generated += ' '
    context_generated.strip()
    train.loc[raw_cnt+i-failure] = {'anchor':train.loc[i]['anchor'],'target':train.loc[i]['target'],\
                            'score':train.loc[i]['score'],'context_text':context_generated}

0
5000
10000
15000
20000
25000
30000
35000


In [9]:
train.loc[46211]

anchor             display object
target               process user
score                         0.5
context_text    PHYSICS HOROLOGY 
Name: 46211, dtype: object

In [10]:
train.loc[46211-raw_cnt]

anchor             display object
target               process user
score                         0.5
context_text    PHYSICS. HOROLOGY
Name: 9738, dtype: object

In [11]:
failure

0

In [13]:
len(train)

72946

In [14]:
train.to_csv('train_shuffled.csv',index=False)

In [11]:
train_shuffled = pd.read_csv('./data/train_shuffled.csv')

In [12]:
train_shuffled

,anchor,target,score,context_text
0,abatement,abatement of pollution,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
1,abatement,act of abating,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
2,abatement,active catalyst,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
3,abatement,eliminating process,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
4,abatement,forest region,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
...,...,...,...,...
72941,wood article,wooden article,1.00,TRANSPORTING PERFORMING OPERATIONS DECORATIVE ...
72942,wood article,wooden box,0.50,TRANSPORTING PERFORMING OPERATIONS DECORATIVE ...
72943,wood article,wooden handle,0.50,TRANSPORTING PERFORMING OPERATIONS DECORATIVE ...
72944,wood article,wooden material,0.75,DECORATIVE ARTS TRANSPORTING PERFORMING OPERAT...


In [13]:
raw_cnt = train_shuffled.shape[0]

In [14]:
for i in range(int(raw_cnt/2)):
    if i%5000 == 0:
        print(i)
    con_words = train_shuffled.loc[i]['context_text'].split(' ')
    cnt_con_words = len(con_words)
    for k in range(cnt_con_words):
        rdt = random.randint(1, 999)
        if rdt%2 == 0:
            con_words[k] = '[MASK]'
    context_generated = ''
    for j in range(cnt_con_words):
        context_generated += con_words[j]
        context_generated += ' '
    context_generated.strip()
    train_shuffled.loc[raw_cnt+i] = {'anchor':train_shuffled.loc[i]['anchor'],'target':train_shuffled.loc[i]['target'],\
                            'score':train_shuffled.loc[i]['score'],'context_text':context_generated}

0
5000
10000
15000
20000
25000
30000
35000


In [15]:
train_shuffled

,anchor,target,score,context_text
0,abatement,abatement of pollution,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
1,abatement,act of abating,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
2,abatement,active catalyst,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
3,abatement,eliminating process,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
4,abatement,forest region,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
...,...,...,...,...
109414,wood article,wooden article,1.00,PERFORMING [MASK] TRANSPORTING. [MASK] [MASK]
109415,wood article,wooden box,0.50,PERFORMING OPERATIONS; [MASK] DECORATIVE [MASK]
109416,wood article,wooden handle,0.50,PERFORMING OPERATIONS; [MASK] [MASK] ARTS
109417,wood article,wooden material,0.75,PERFORMING [MASK] TRANSPORTING. [MASK] ARTS


In [17]:
train_shuffled.to_csv('./data/train_shuffled_masked.csv',index=False)